In [ ]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

In [ ]:
# HELPER FUNCTIONS
group_number = 14

# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index


# Find contours in the image
def check_contours_new(frame):

    print('Checking image:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)

    print(largest_area)

    return contours, largest_contour_index

In [ ]:
# First task: obtain x and z coordinates of the obstacle (create bounding box from the contour arrays)

camera = 0
color = (0, 0, 0) # black rgb value
t = time.time()
elapsed = time.time() - t


cap = cv2.VideoCapture(camera)
while(cap.isOpened()):

    ret, frame = cap.read()

    elapsed = time.time() - t
    if(elapsed > 5.0):

        print('Capturing.....')

        if ret:
            contours, largest_contour_index = check_contours_new(frame)
            # make bounding box
            x,y,w,h = cv2.boundingRect(contours[largest_contour_index])
            cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), color, thickness=2)
            print(int(x), int(y),int(w),int(h))
            coords = str(int(x))+","+str(int(y)) 
            cv2.putText(frame, coords, (int(x), int(y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            cv2.imshow('img', frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
    if(elapsed > 60.0):
        break

cap.release()